In [2]:
import tensorflow as tf 
from tensorflow import keras as keras
from tensorflow.python.keras import backend as K
import os
import numpy as np
import datetime
import importlib

import SatCallbacks
import Clone_Model
import rsc

In [3]:
checkpoint_path = "./cloning/checkpoints/cloning/default_checkpoint.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
ckpt_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path, save_weights_only=True, verbose=1)

log_dir_clone = "./cloning/logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir_clone, histogram_freq=1)

In [7]:
#      CLONE MODEL EAGER ENABLED
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)

if not tf.executing_eagerly(): tf.compat.v1.enable_eager_execution()

K.clear_session()

batch_size = 3

train, test = rsc.get_titanic_dataset()


model = rsc.get_model_slim()

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=keras.losses.BinaryCrossentropy(),
    run_eagerly = tf.executing_eagerly()
    )


history = model.fit(train,
                    batch_size=batch_size,
                    epochs=1, 
                    steps_per_epoch=3, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback])


clone = Clone_Model.satify_model(model)

clone.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.BinaryCrossentropy(),
              run_eagerly = tf.executing_eagerly())

model.summary()
clone.summary()

#sat_cb = Clone_Model.sat_results()
sat_cb = SatCallbacks.sat_results()

history = clone.fit(train,
                    batch_size=batch_size,
                    epochs = 1,
                    steps_per_epoch=3,
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback, sat_cb])
                
    

3/3 [==============================] - 0s 19ms/step - loss: 0.6928 - val_loss: 0.6920
input_shape: [None, 3]
layer_width: (3,)
input_shape: [None, 3]
layer_width: (3,)
input_shape: [None, 2]
layer_width: (2,)
input_shape: [None, 2]
layer_width: (2,)
input_shape: [None, 2]
layer_width: (2,)
Layer InputLayer
Layer mydense
Layer mydense
Layer mydense
Layer mydense
Layer mydense
Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
model_input (InputLayer)     [(None, 4)]               0         
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 15        
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 12        
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 8         
_____________

ValueError: Shapes (10, 3) and (20, 3) are incompatible

In [4]:
#      TRAIN MODEL EAGER ENABLED MNIST CLASSIFICATION
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)

if not tf.executing_eagerly(): tf.compat.v1.enable_eager_execution()

K.clear_session()

batch_size = 20

train, test = rsc.get_mnist()


model = rsc.get_model_unitlist(hidden_layers_spec=[128])

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])



history = model.fit(train[0],
                    train[1],
                    batch_size=batch_size,
                    epochs=1, 
                    steps_per_epoch=None, 
                    validation_data=test, 
                    validation_steps=1, 
                    callbacks=[tensorboard_callback])



3000/3000 [==============================] - 16s 5ms/step - loss: 2.0855 - accuracy: 0.8489 - val_loss: 0.0344 - val_accuracy: 1.0000


In [5]:
#      CLONE MODEL EAGER ENABLED MNIST CLASSIFICATION
###########################################

importlib.reload(rsc)
importlib.reload(Clone_Model)
importlib.reload(SatCallbacks)

print(tf.executing_eagerly())

clone = Clone_Model.satify_model(model)

"""clone.compile(optimizer=keras.optimizers.Adam(),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              run_eagerly = tf.executing_eagerly())
"""

clone.summary()
new_cb = SatCallbacks.sat_logger()

#sat_cb = Clone_Model.sat_results()
sat_cb = SatCallbacks.sat_results()

#batch size is 10 so incidentally no problem
history = clone.fit(train[0],
                    train[1],
                    batch_size=11,
                    epochs = 2,
                    steps_per_epoch=5,
                    validation_data=test, 
                    validation_steps=2, 
                    callbacks=[new_cb])
              

True
Model output_shape: (None, 10)
input_shape: [None, 128]
layer_width: (128,)
SatLayer input shape (None, 128) not matching current_activation property shape (0, 128)
batch_size initialized. current shape is: Tensor("dense_2/Shape:0", shape=(1,), dtype=int32)
input_shape: [None, 10]
layer_width: (10,)
SatLayer input shape (None, 10) not matching current_activation property shape (0, 10)
batch_size initialized. current shape is: Tensor("dense_1_1/Shape:0", shape=(1,), dtype=int32)
Layer Flatten
Layer mydense
Layer mydense
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (mydense)              (None, 128)               116993    
_________________________________________________________________
dense_1 (mydense)            (None, 10)               

InvalidArgumentError: Incompatible shapes: [0,128] vs. [11,128] [Op:Equal]

In [62]:
in_shape = model.layers[0].input_shape[-2:]
print(in_shape)
np.ones(shape=in_shape).shape

(28, 28)


(28, 28)

In [81]:
#train[0].shape
#train[1]
#clone.layers[1].sat_layer.get_config()
clone.layers[1].sat_layer._batch_input_shape

AttributeError: 'sat_layer' object has no attribute '_batch_input_shape'

In [45]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:XLA_CPU:0', device_type='XLA_CPU')]

In [44]:
tf.distribute.get_strategy()

In [46]:
#(None, 21, 5)[1:]*2
#np.zeros(shape=(None))
importlib.reload(rsc)
train_b, test_b = rsc.get_titanic_dataset(batch_size=3)
print(train_b)
print(model(np.array([[1,2,3,4] ])))

for ix, x in enumerate(train_b):
    print(x)

<BatchDataset shapes: ((None, None, 4), (None, None, 2)), types: (tf.float32, tf.int32)>
tf.Tensor([[0.69211704 0.3078829 ]], shape=(1, 2), dtype=float32)
(<tf.Tensor: shape=(3, 20, 4), dtype=float32, numpy=
array([[[ 61.    ,   0.    ,   0.    ,  32.3208],
        [ 38.    ,   0.    ,   1.    , 153.4625],
        [ 36.    ,   0.    ,   0.    ,  26.3875],
        [ 28.    ,   0.    ,   0.    ,  26.55  ],
        [ 22.    ,   0.    ,   0.    ,  10.5167],
        [ 28.    ,   3.    ,   1.    ,  25.4667],
        [ 28.    ,   0.    ,   0.    ,   7.8958],
        [ 30.    ,   0.    ,   0.    ,   7.8958],
        [ 34.    ,   1.    ,   0.    ,  21.    ],
        [  3.    ,   4.    ,   2.    ,  31.3875],
        [ 31.    ,   1.    ,   0.    ,  57.    ],
        [ 16.    ,   0.    ,   0.    ,   8.05  ],
        [ 28.    ,   3.    ,   1.    ,  25.4667],
        [ 39.    ,   0.    ,   0.    ,   0.    ],
        [ 28.    ,   0.    ,   0.    , 227.525 ],
        [ 19.    ,   0.    ,   0.    ,   7

InvalidArgumentError: Cannot add tensor to the batch: number of elements does not match. Shapes are: [tensor]: [7,4], [batch]: [20,4]